# NLP Analysis of Employee Engagement Survey

## Summary
This project aims to apply NLP and data visualization techniques to automate the insight extraction of employee surveys and discover potential issues and highlights in the survey responses to help HR better design employee experience.

## Data
The employee survey responses in the company.

## Project Sections
 - Snowflake Data Preprocessing & Loading
    - Sentiment Score and Summarization Generation with Snowflake LLM Functions
 - Survey Respondent Profile Analysis
 - Response Sentiment Analysis
 - Word Cloud of Frequent Keywords
 - Topic Modeling with BERTopic
 - Topic Grouping into Broad Categories
 - Targeted Sentiment Analysis around Topic Categories


 ## Author and Platform
Yezi Liu conducted this project in AWS with the support from teammates.

 _**(Important Note: for the purpose of data & company privacy, the code below is modified to contain some pseudo-code, but the workflows and logic are the same)**_

In [ ]:
# running this cell may make changes to your environment
# !pip install -r requirements.txt

## Install Packages

In [ ]:
# Helper functions
# ** Note: This utilities file with all the helper functions isn't included in this notebook or repo for the protection of the company privacy.**
from utilities import *

# Snowpark
import snowflake.snowpark.functions as F
from snowflake.snowpark.functions import when, date_part

# Cortex Functions
import snowflake.cortex  as cortex
from snowflake.snowpark import Session

# Python packages
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap, LinearSegmentedColormap
import numpy as np
import os
import re
import string
import nltk
from nltk.corpus import stopwords
from nltk.corpus import wordnet
from wordcloud import WordCloud
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import sent_tokenize
from nltk.probability import FreqDist
from collections import Counter
from bertopic import BERTopic

nltk.download('wordnet')
nltk.download('punkt')
nltk.download('stopwords')

## Snowflake Data Preprocessing and Loading

In [ ]:
user = "user_name"
input_db = "database_name"
input_schema = "schema_name"
# define connectors to snowflake (make sure to use correct role)
cur, conn = connect_to_snowflake(user, input_db, input_schema)

In [ ]:
# Conducted basic preprocessing to survey responses(remove stop words, punctuations, numbers,and lowercase, etc)

# Added question context as a prefix to the responses for survey questions to help
# generate accurate sentiment scores for questions' responses

# Applied Snowflake functions to generate sentiment scores and summaries for each response

sql_query = """
WITH stopwords AS (
    SELECT 'a' AS stopword UNION ALL
    SELECT 'an' UNION ALL
    SELECT 'and' UNION ALL
    SELECT 'are' UNION ALL
    SELECT 'as' UNION ALL
    SELECT 'at' UNION ALL
    SELECT 'be' UNION ALL
    SELECT 'by' UNION ALL
    SELECT 'for' UNION ALL
    SELECT 'has' UNION ALL
    SELECT 'he' UNION ALL
    SELECT 'in' UNION ALL
    SELECT 'is' UNION ALL
    SELECT 'it' UNION ALL
    SELECT 'its' UNION ALL
    SELECT 'of' UNION ALL
    SELECT 'on' UNION ALL
    SELECT 'that' UNION ALL
    SELECT 'the' UNION ALL
    SELECT 'was' UNION ALL
    SELECT 'were' UNION ALL
    SELECT 'will' UNION ALL
    SELECT 'with'
),
stopword_pattern AS (
    SELECT LISTAGG('\\b' || stopword || '\\b', '|') WITHIN GROUP (ORDER BY stopword) AS pattern
    FROM stopwords
),
cleaned_comments AS (
    SELECT
        col1,
        col2,
        col3,
        col4,
        REGEXP_REPLACE(
            REGEXP_REPLACE(
                REGEXP_REPLACE(
                    LOWER(comment), '\\b(' || (SELECT pattern FROM stopword_pattern) || ')\\b', ''
                ), '[[:punct:]]', ''
            ), '[0-9]', ''
        ) AS cleaned_comment
    FROM
        data_path
)
SELECT
    col1,
    col2,
    col3,
    col4,
    CASE
        WHEN question = Q1 THEN
            'Q1 prefix' || cleaned_comment
        ELSE
            cleaned_comment
    END AS comment,
    SNOWFLAKE.CORTEX.SUMMARIZE(
        CASE
            WHEN question = Q1 THEN
                'Q1 prefix ' || cleaned_comment
            ELSE
                cleaned_comment
        END
    ) AS summary,
    SNOWFLAKE.CORTEX.SENTIMENT(
        CASE
            WHEN question = Q1 THEN
                'Q1 prefix ' || cleaned_comment
            ELSE
                cleaned_comment
        END
    ) AS sentiment
FROM
    cleaned_comments
"""

In [ ]:
# Bring the data to local pandas
df = get_data(sql_query, cur)

In [ ]:
# Identify rows where the comment contains exactly the word "nothing"
contains_nothing = df['comment'].str.contains(r'\bnothing\b', case=False, na=False)

# Identify rows where the comment length is less than or equal to 15 words
short_comments = df['comment'].str.split().str.len() <= 15

nothing_df = df[contains_nothing & short_comments]

# Exclude these short "nothing" responses out of future analysis, assuming there isn't useful information in those responses
df = df.drop(nothing_df.index)

## Survey Respondents Profile

In [ ]:
# Map the questions to their short forms
question_map = {
    'Question 1 Conetent': 'Q1',
    'Question 2 Content': 'Q2'
}

df['question_short'] = df['question'].map(question_map)

### Distribution of Respondents by Subsidiary and Question

In [ ]:
total_count = len(df)

# Create the count plot
plt.figure(figsize=(8, 6))
ax = sns.countplot(x='subsidiary', hue='question_short', data=df, palette='viridis')

# Calculate percentages and update y-tick labels
y_ticks = ax.get_yticks()
y_labels = [f'{(y / total_count) * 100:.1f}%' for y in y_ticks if y > 0]  # Filter out zero ticks
ax.set_yticklabels(y_labels)

# Add percentages above the bars
for p in ax.patches:
    if p.get_height() > 0:  # Only annotate bars with height greater than zero
        percentage = (p.get_height() / total_count) * 100
        ax.annotate(f'{percentage:.1f}%',
                    (p.get_x() + p.get_width() / 2., p.get_height()),
                    ha='center', va='bottom', fontsize=10, color='black')

plt.title('Distribution of Respondents by Subsidiary and Question')
plt.xlabel('Subsidiary')
plt.ylabel('Percentage')
plt.legend(title='Question')

plt.show()

### Distribution of Respondents by Region and Question

In [ ]:
total_count = len(df)

# Create the count plot
plt.figure(figsize=(8, 6))
ax = sns.countplot(x='region', hue='question_short', data=df, palette='plasma')

# Calculate percentages for each bar and update y-tick labels
y_ticks = ax.get_yticks()
y_labels = [f'{(y / total_count) * 100:.1f}%' for y in y_ticks]
ax.set_yticklabels(y_labels)

# Add percentages above the bars
for p in ax.patches:
    if p.get_height() > 0:  # Only annotate bars with height greater than zero
        percentage = (p.get_height() / total_count) * 100
        ax.annotate(f'{percentage:.1f}%',
                    (p.get_x() + p.get_width() / 2., p.get_height()),
                    ha='center', va='bottom', fontsize=10, color='black')

plt.title('Distribution of Respondents by Region and Question')
plt.xlabel('Region')
plt.ylabel('Percentage')
plt.legend(title='Question')

plt.show()

### Distribution of Respondents by Question

In [ ]:
total_count = len(df)

# Create the count plot
plt.figure(figsize=(6, 4))
ax = sns.countplot(y='question', data=df, palette='cubehelix')

# Calculate percentages and update x-tick labels
x_ticks = ax.get_xticks()
x_labels = [f'{(x / total_count) * 100:.1f}%' for x in x_ticks]
ax.set_xticklabels(x_labels)

# Add percentages next to the bars
for p in ax.patches:
    percentage = (p.get_width() / total_count) * 100
    ax.annotate(f'{percentage:.1f}%',
                (p.get_width(), p.get_y() + p.get_height() / 2.),
                ha='left', va='center', fontsize=10, color='black')

plt.title('Distribution of Respondents by Question')
plt.xlabel('Percentage')
plt.ylabel('Question')

plt.show()

### Cross-Tabulation of Subsidiary, Region, and Question Breakdown

In [ ]:
subsidiaries = df['subsidiary'].unique()
regions = df['region'].unique()

# Create a MultiIndex from all combinations of subsidiaries and regions
index = pd.MultiIndex.from_product([subsidiaries, regions], names=['subsidiary', 'region'])

# Create a crosstab for actual counts, reindex to include all combinations, and fill missing values with 0
cross_tab_counts = pd.crosstab(index=[df['subsidiary'], df['region']], columns=df['question_short']).reindex(index, fill_value=0)

cross_tab_counts['Total'] = cross_tab_counts.sum(axis=1)

# Calculate the percentage breakdown within each subgroup
cross_tab_percentage = cross_tab_counts.drop(columns='Total').div(cross_tab_counts['Total'].replace({0: np.nan}), axis=0)

plt.figure(figsize=(14, 8))
ax = sns.heatmap(cross_tab_counts, annot=True, cmap='coolwarm_r', fmt='d', cbar=True, linewidths=.5, linecolor='gray')

# Overlay percentage annotations on the right side of the counts
for y in range(cross_tab_percentage.shape[0]):
    for x in range(cross_tab_percentage.shape[1]):
        percentage = round(cross_tab_percentage.iloc[y, x] * 100) if not np.isnan(cross_tab_percentage.iloc[y, x]) else 0
        ax.text(x + 0.75, y + 0.5, f'({percentage}%)',
                horizontalalignment='center', verticalalignment='center',
                fontsize=10, fontweight='bold')

# Format the counts in the heatmap
for text in ax.texts:
    text.set_fontsize(10)
    text.set_fontweight('bold')

plt.title('Cross-Tabulation of Subsidiary, Region, and Question Breakdown')
plt.xlabel('Question and Total')
plt.ylabel('Subsidiary and Region')
plt.xticks(rotation=0)
plt.yticks(rotation=0)
plt.show()

### Cross-Tabulation of Question and Region

In [ ]:
plt.figure(figsize=(4, 2))
cross_tab = pd.crosstab(df['question_short'], df['region'])
sns.heatmap(cross_tab, annot=True, cmap='magma', fmt='d')
plt.title('Cross-Tabulation of Question and Region')
plt.xlabel('Region')
plt.ylabel('Question')
plt.show()

### Cross-Tabulation of Question and Subsidiary

In [ ]:
plt.figure(figsize=(4, 2))
cross_tab = pd.crosstab(df['question_short'], df['subsidiary'])
sns.heatmap(cross_tab, annot=True, cmap='magma', fmt='d')
plt.title('Cross-Tabulation of Question and Subsidiary')
plt.xlabel('Region')
plt.ylabel('Question')
plt.show()

## Sentiment Analysis

### Overall Sentiment Distribution

In [ ]:
plt.figure(figsize=(8, 6))
plt.hist(df['sentiment'], bins=30, edgecolor='k', alpha=0.7)
plt.title('Overall Sentiment Distribution', fontsize=12)
plt.xlabel('Sentiment', fontsize=12)
plt.ylabel('Frequency', fontsize=12)
plt.show()

### Sentiment Distribution by Question

In [ ]:
q1_df = df[df['question'] == "Q1 Content"]
q2_df = df[df['question'] == "Q2 Content"]

In [ ]:
plt.figure(figsize=(8, 6))

plt.hist(q1_df['sentiment'], bins=30, edgecolor='k', alpha=0.7, color='skyblue', label="Q1 Content")

plt.hist(q2_df['sentiment'], bins=30, edgecolor='k', color='salmon', label="Q2 Content", alpha=0.5)

plt.title('Sentiment Distribution by Question', fontsize=12)
plt.xlabel('Sentiment', fontsize=12)
plt.ylabel('Frequency', fontsize=12)

plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
plt.legend(fontsize=10, loc='upper left')

plt.show()

### Box Plot of Sentiment by Question

In [ ]:
sns.boxplot(x="sentiment", y = "question", data = df)

### Average Sentiment Score by Region

In [ ]:
avg_sentiment_by_region = df.groupby('region')['sentiment'].mean().reset_index()

pivot_table = avg_sentiment_by_region.pivot_table(index='region', values='sentiment')

plt.figure(figsize=(6, 4))
sns.heatmap(pivot_table, annot=True, cmap='coolwarm', center=0)
plt.title('Average Sentiment Score by Region', fontsize = 12)
plt.xlabel('Region', fontsize = 12)
plt.ylabel('Sentiment Score', fontsize = 12)
plt.show()

### Sentiment Proportions by Region and Question

In [ ]:
# Create a FacetGrid for the sentiment distribution by region
g = sns.FacetGrid(df, col="region", col_wrap=5, height=5, sharex=True, sharey=True)

g.map_dataframe(plot_hist)

g.set_axis_labels("Sentiment", "Percentage")
g.set_titles(col_template="{col_name} Region")

for ax in g.axes.flat:
    handles, labels = ax.get_legend_handles_labels()
    if handles:
        ax.legend(handles=handles, labels=labels, loc='upper right', fontsize=10)
        break

plt.show()

### Box Plot of Sentiment by Question and Region

In [ ]:
sns.boxplot(x="sentiment", y="question", hue = "region", data=df)

plt.legend(title='Region', fontsize='small', title_fontsize='medium', loc='upper right', bbox_to_anchor=(1.25, 1))

plt.title('Box Plot of Sentiment by Question and Region')
plt.xlabel('Sentiment')
plt.ylabel('Question')

plt.show()

### Average Sentiment Score by Subsidiary

In [ ]:
avg_sentiment_by_subsidiary = df.groupby('subsidiary')['sentiment'].mean().reset_index()

pivot_table = avg_sentiment_by_subsidiary.pivot_table(index='subsidiary', values='sentiment')

plt.figure(figsize=(6, 4))
sns.heatmap(pivot_table, annot=True, cmap='coolwarm', center=0)
plt.title('Average Sentiment Score by Subsidiary', fontsize = 12)
plt.xlabel('Subsidiary', fontsize = 12)
plt.ylabel('Sentiment Score', fontsize = 12)
plt.show()

### Sentiment Proportions by Subsidiary and Question

In [ ]:
g = sns.FacetGrid(df, col="subsidiary", col_wrap=5, height=5, sharex=True, sharey=True)

g.map_dataframe(plot_hist)

g.set_axis_labels("Sentiment", "Percentage")
g.set_titles(col_template="{col_name} Subsidiary")

for ax in g.axes.flat:
    handles, labels = ax.get_legend_handles_labels()
    if handles:
        ax.legend(handles=handles, labels=labels, loc='upper right', fontsize=10)
        break

plt.show()

### Box Plot of Sentiment by Question and Subsidiary

In [ ]:
sns.boxplot(x="sentiment", y = "question", hue = "subsidiary", data = df)

plt.legend(title='Subsidiary', fontsize='small', title_fontsize='medium', loc='upper right', bbox_to_anchor=(1.25, 1))

plt.title('Box Plot of Sentiment by Question and Subsidiary')
plt.xlabel('Sentiment')
plt.ylabel('Question')

plt.show()

### Proportion of Sentiment Categories Within Each Subsidiary

In [ ]:
df['sentiment_category'] = df['sentiment'].apply(categorize_sentiment)

In [ ]:
# Compute the proportion of sentiment categories by subsidiary
sentiment_proportion_by_subsidiary = df.groupby(['subsidiary', 'sentiment_category']).size().unstack(fill_value=0)
sentiment_proportion_by_subsidiary = sentiment_proportion_by_subsidiary.div(sentiment_proportion_by_subsidiary.sum(axis=1), axis=0)

sentiment_proportion_by_subsidiary.plot(kind='bar', stacked=True, figsize=(8, 4), colormap='viridis')
plt.title('Proportion of Sentiment Categories Within Each Subsidiary',fontsize=12)
plt.xlabel('Subsidiary', fontsize=12)
plt.ylabel('Proportion', fontsize=12)
plt.legend(title='Sentiment Category', bbox_to_anchor=(1.05, 1), loc='upper left', fontsize='small', title_fontsize='small')
plt.show()

### Proportion of Sentiment Categories Within Each Region

In [ ]:
# Compute the proportion of sentiment categories by region
sentiment_proportion_by_region = df.groupby(['region', 'sentiment_category']).size().unstack(fill_value=0)
sentiment_proportion_by_region = sentiment_proportion_by_region.div(sentiment_proportion_by_region.sum(axis=1), axis=0)

sentiment_proportion_by_region.plot(kind='bar', stacked=True, figsize=(8, 4), colormap='viridis')
plt.title('Proportion of Sentiment Categories Within Each Region',fontsize=12)
plt.xlabel('Region', fontsize=12)
plt.ylabel('Proportion', fontsize=12)
plt.legend(title='Sentiment Category', bbox_to_anchor=(1.05, 1), loc='upper left', fontsize='small', title_fontsize='small')
plt.show()

### Cross-Tabulation of Average Sentiment Scores by Subsidiary, Region, and Question

In [ ]:
subsidiaries = df['subsidiary'].unique()
regions = df['region'].unique()
questions = df['question_short'].unique()

all_combinations = pd.DataFrame([(s, r, q) for s in subsidiaries for r in regions for q in questions],
                                columns=['subsidiary', 'region', 'question_short'])

average_sentiment = df.groupby(['subsidiary', 'region', 'question_short'])['sentiment'].mean().reset_index()

average_sentiment_complete = pd.merge(all_combinations, average_sentiment,
                                      on=['subsidiary', 'region', 'question_short'], how='left')

average_sentiment_complete['sentiment'].fillna(0, inplace=True)

pivot_table = average_sentiment_complete.pivot_table(index=['subsidiary', 'region'], columns='question_short', values='sentiment')

plt.figure(figsize=(8, 6))
ax = sns.heatmap(pivot_table, annot=True, cmap='coolwarm_r', linewidths=0.5, linecolor='gray', cbar_kws={'label': 'Average Sentiment Score'})

plt.title('Average Sentiment Scores by Subsidiary, Region, and Question')
plt.xlabel('Question')
plt.ylabel('Subsidiary and Region')
plt.xticks(rotation=0)
plt.yticks(rotation=0)
plt.show()

## Frequent Key Words via Word Cloud

### Preprocessing Data to Prepare for Word Cloud

In [ ]:
stop_words = set(stopwords.words('english'))
custom_stopwords = {"Add all the customized stop words based on specific context"}
stop_words.update(custom_stopwords)

In [ ]:
df['cleaned_comment'] = df['comment'].apply(clean_text)

### Word Cloud By Question & Polarized Sentiment Categories

In [ ]:
# Generate word clouds for each question and sentiment category
questions = df['question'].unique()
sentiment_categories = ['Strong Positive', 'Other', 'Strong Negative']

for question in questions:
    for sentiment in sentiment_categories:
        # Filter the DataFrame based on question and sentiment category
        filtered_df = df[(df['question'] == question) & (df['sentiment_category'] == sentiment)]
        text = ' '.join(filtered_df['cleaned_comment'])
        if text:
            wordcloud = WordCloud(stopwords=stop_words, width=800, height=400, background_color='white').generate(text)
            plt.figure(figsize=(8, 6))
            plt.imshow(wordcloud, interpolation='bilinear')
            plt.axis('off')
            plt.title(f'Word Cloud for "{question}" ({sentiment} Sentiment)')
            plt.show()

## Topic Modeling with BERTopic

### Data Preprocessing for Topic Modeling

In [ ]:
docs = clean_and_extract_docs_for_bertopic(df, text_column='comment', stop_words_extra=list(custom_stopwords))

### Fit Model & Analyze Topics

**Note: BERTopic is stochastic so the topics might differ across runs. This is mostly due to the stocastisch nature of UMAP.**

In [ ]:
topic_model = BERTopic(language="english", min_topic_size=60, n_gram_range=(1,2), calculate_probabilities=True, verbose=True)
topics, probs = topic_model.fit_transform(docs)

**Note: The topic `-1` refers to all outliers and should typically be ignored.**

In [ ]:
freq = topic_model.get_topic_info()

print(f"Topics found: {freq.shape[0]}")
freq.head(10)

In [ ]:
# This graph shows the hierarchical relationships of all topics and how they are related to each other
topic_model.visualize_hierarchy()

In [ ]:
# This graph visualizes the selected terms for each topic. The numbers on the horizontal axis are
# c-TF-IDF scores for us to compare between and within topics
topic_model.visualize_barchart(top_n_topics=49)

In [ ]:
# Save the model
topic_model.save('bert_topics_model')

**Note: The model and its internal settings can easily be saved. The documents and embeddings will not be saved. However, UMAP and HDBSCAN will be saved. As a result, when we reload the model and refit on the same documents, it will
generate different number of topics each time even if the original topics are saved and loaded with the model.
Therefore, we cannot replicate these topics results unless we explicitly save those topices and the topic-document relationships.**

In [ ]:
# Load the model
topic_model = BERTopic.load('bert_topics_model')
freq = topic_model.get_topic_info()

print(f"Topics found: {freq.shape[0]}")
freq.head(10)

### Add Topics Results to Our Dataframe

In [ ]:
# For each document/row/survey response, only the most significant topic number from all the relevant
# topics to that document will be saved, so there is 1 to 1 relationship between document and topic
# in our dataframe.
df['bert_topic'] = topics

### Classify Topics into Broader Categories and Select 10 Categories Based on Stakeholder's Areas of Interests

In [ ]:
# Define the mapping of topics to broader sections/categories
topic_to_category = {
    1: 'Leadership',
    12: 'Leadership',
    2: 'Work Life Balance',
    0: 'Teamwork',
    # Omit all the rest mappings
}

# Map topics to categories
df['category'] = df['bert_topic'].map(topic_to_category)

In [ ]:
category_counts = df['category'].value_counts(dropna=False)
print(category_counts)

**Note: RTO is another important area that the HR stakeholder is interested in, but it didn't appear in the selected terms of all the topics. So I decided to search for it among the rest of the survey respones that hasn't been mapped to a specific category.**

In [ ]:
# Filter out the rows where the comment contains the RTO keywords
nan_category_df = df[df['category'].isnull()]
rto_keywords = ["add all the key words related to this topic"]
rto_keyword_mask = nan_category_df['comment'].str.contains('|'.join(rto_keywords), case=False, na=False)
rto_df = nan_category_df[rto_keyword_mask]

In [ ]:
rto_mask = df.index.isin(rto_df.index)

# Update the 'category' column for these rows to 'RTO'
df.loc[rto_mask, 'category'] = 'RTO'

df['category'].value_counts(dropna=False)

## Sentiment Analysis for Categories/Topics

### Compare Average Sentiment per Category and Visualize Sentiment Distributions across Category

In [ ]:
avg_sentiment = df.groupby('category')['sentiment'].mean().reset_index()

print("Average Sentiment for Each Topic:")
print(avg_sentiment)

unique_sections = df['category'].dropna().unique()
for section in unique_sections:
    plt.figure(figsize=(6, 4))
    sns.histplot(df[df['category'] == section]['sentiment'], bins=30, kde=True)
    plt.title(f'Sentiment Distribution for {section}')
    plt.xlabel('Sentiment')
    plt.ylabel('Frequency')
    plt.show()

### Compare Positive and Negative Sentiment Proportion per Category

In [ ]:
# Calculate the total number of sentiment scores for each category
total_sentiment = df.groupby('category')['sentiment'].count().reset_index(name='total_count')

# Calculate the number of positive sentiment scores for each category
positive_sentiment = df[df['sentiment'] > 0].groupby('category')['sentiment'].count().reset_index(name='positive_count')

# Calculate the number of negative sentiment scores for each category
negative_sentiment = df[df['sentiment'] < 0].groupby('category')['sentiment'].count().reset_index(name='negative_count')

sentiment_counts = total_sentiment.merge(positive_sentiment, on='category', how='left').merge(negative_sentiment, on='category', how='left')

# Fill NaN values with 0 (in case there are categories with no positive or negative sentiments)
sentiment_counts = sentiment_counts.fillna(0)

sentiment_counts['positive_percentage'] = (sentiment_counts['positive_count'] / sentiment_counts['total_count']) * 100
sentiment_counts['negative_percentage'] = (sentiment_counts['negative_count'] / sentiment_counts['total_count']) * 100

final_table = sentiment_counts[['category', 'positive_percentage', 'negative_percentage']]

final_table

### Counts by Category and Subsidiary

In [ ]:
# Create a crosstab for counts by category and subsidiary
category_subsidiary_counts = pd.crosstab(df['category'], df['subsidiary'])

print("Counts by Category and Subsidiary:")
print(category_subsidiary_counts)

### Counts by Category and Region

In [ ]:
# Create a crosstab for counts by category and region
category_region_counts = pd.crosstab(df['category'], df['region'])

print("Counts by Category and Region:")
print(category_region_counts)

In [ ]:
sentiment_mapping = {
    'Strong Negative Sentiment': 1,
    'Moderate Negative Sentiment': 2,
    'Mild Negative Sentiment': 3,
    'Mild Positive Sentiment': 4,
    'Moderate Positive Sentiment': 5,
    'Strong Positive Sentiment': 6,
}

full_cmap = sns.color_palette([
    "#d73027",  # Strong Negative - Red
    "#f46d43",  # Moderate Negative - Red-Orange
    "#fdae61",  # Mild Negative - Orange-Yellow
    "#fee08b",  # Mild Positive - Yellow-White
    "#abd9e9",  # Moderate Positive - Light Blue
    "#4575b4",  # Strong Positive - Blue
])

### Visualize Average Sentiment by Category and Subsidiary in Various Ways

In [ ]:
# Remove rows with NaN in the 'category' column
df = df.dropna(subset=['category'])

# Create the pivot table for average sentiment
pivot_table = pd.pivot_table(
    df,
    values='sentiment',
    index='category',
    columns='subsidiary',
    aggfunc='mean'
)

# Filter out groups with fewer than 25 counts
filtered_pivot_table = pivot_table.where(category_subsidiary_counts >= 25)

print(filtered_pivot_table)

plt.figure(figsize=(10, 8))
sns.heatmap(filtered_pivot_table, annot=True, cmap='coolwarm_r', center=0, linewidths=.5, fmt='.2f')
plt.title('Average Sentiment by Category and Subsidiary')
plt.show()


# Apply the categorization function to the entire pivot table
categorized_pivot_table = filtered_pivot_table.applymap(classify_sentiment)

# Apply the mapping to the categorized pivot table for visualization
visualization_pivot_table = categorized_pivot_table.replace(sentiment_mapping)

# Manually map each sentiment number to its corresponding color
full_color_dict = {val: full_cmap[i] for i, val in enumerate(sentiment_mapping.values())}

custom_cmap = [full_color_dict[val] for val in sorted(sentiment_mapping.values()) if val in visualization_pivot_table.values]

plt.figure(figsize=(10, 8))
sns.heatmap(visualization_pivot_table, cmap=custom_cmap, linewidths=.5,
            cbar_kws={'ticks': sorted(sentiment_mapping.values())})

colorbar = plt.gca().collections[0].colorbar
colorbar.set_ticklabels([k for k, v in sorted(sentiment_mapping.items(), key=lambda item: item[1])])

plt.title('Categorized Sentiment by Category and Subsidiary')
plt.show()

**Note: The blank cells indicate that there are fewer than 25 counts in those subgroups, and they are excluded from the analysis due to the small size.**

### Visualize Average Sentiment by Category and Region in Various Ways

In [ ]:
# Remove rows with NaN in the 'category' column
df = df.dropna(subset=['category'])

# Create the pivot table for average sentiment
pivot_table = pd.pivot_table(
    df,
    values='sentiment',
    index='category',
    columns='region',
    aggfunc='mean'
)

# Filter out groups with fewer than 25 counts
filtered_pivot_table = pivot_table.where(category_region_counts >= 25)
print(filtered_pivot_table)

# Visualize the pivot table as a heatmap
plt.figure(figsize=(10, 8))
sns.heatmap(filtered_pivot_table, annot=True, cmap='coolwarm_r', center=0, linewidths=.5, fmt='.2f')
plt.title('Average Sentiment by Category and Region')
plt.show()

# Apply the categorization function to the entire filtered pivot table
categorized_pivot_table = filtered_pivot_table.applymap(classify_sentiment)

# Apply the mapping to the categorized pivot table for visualization
visualization_pivot_table = categorized_pivot_table.replace(sentiment_mapping)

# Identify the unique sentiment categories present in the data
present_categories = categorized_pivot_table.stack().unique()

# Create a mapping for the present categories to their numerical values
present_mapping = {cat: sentiment_mapping[cat] for cat in present_categories}

filtered_cmap = [full_cmap[sentiment_mapping[cat] - 1] for cat in sorted(present_categories, key=lambda x: sentiment_mapping[x])]

# Create the heatmap using the filtered color palette
plt.figure(figsize=(10, 8))
sns.heatmap(visualization_pivot_table, cmap=filtered_cmap, linewidths=.5,
            cbar_kws={'ticks': sorted(present_mapping.values())})

# Customizing the colorbar to show only the present categories
colorbar = plt.gca().collections[0].colorbar
colorbar.set_ticklabels(sorted(present_mapping.keys(), key=lambda x: sentiment_mapping[x]))

plt.title('Categorized Sentiment by Category and Region')
plt.show()

**Note: Similarly, the blank cells indicate that there are fewer than 25 counts in those subgroups, and they are excluded from the analysis due to the small size.**